In [1]:
from google.colab import drive
drive.mount('/content/drive')
# /content/drive/MyDrive/Final Project

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import sys
import torch
from torch.utils.data import DataLoader
from torchvision import transforms

# Set working directory (replace 'path/to/your/data' with your actual data path)
import os
os.chdir('/content/drive/MyDrive/FinalProject')
sys.path.append(os.getcwd())


# Print working directory for verification
print(f"Current Working Directory: {os.getcwd()}")

Current Working Directory: /content/drive/MyDrive/FinalProject


In [ ]:
captions_file="/content/drive/MyDrive/FinalProject/data/captions.txt"

In [4]:
!pip install torch torchvision


In [5]:
from dataset import ImageCaptionDataset
from encoder import Encoder
from decoder import DecoderWithAttention


In [ ]:
import csv

# Convert captions.txt from CSV to tab-separated format
input_file = "data/captions.txt"  # Update if necessary
output_file = "data/captions_fixed.txt"  # Adjust output path as needed

with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    reader = csv.reader(infile)  # Default is comma-delimited
    next(reader)  # Skip header
    for row in reader:
        # Ensure the row has at least two columns
        if len(row) >= 2:
            img_id = row[0].strip()  # Image ID
            caption = ",".join(row[1:]).strip()  # Join remaining parts as the caption
            outfile.write(f"{img_id}\t{caption}\n")

print(f"Converted captions saved to {output_file}")



Converted captions saved to data/captions_fixed.txt


In [6]:
with open("data/captions_fixed.txt", "r") as f:
    lines = f.readlines()
    print(f"Total lines: {len(lines)}")
    print(f"First 5 lines: {lines[:5]}")

Total lines: 40455
First 5 lines: ['1000268201_693b08cb0e.jpg\tA child in a pink dress is climbing up a set of stairs in an entry way .\n', '1000268201_693b08cb0e.jpg\tA girl going into a wooden building .\n', '1000268201_693b08cb0e.jpg\tA little girl climbing into a wooden playhouse .\n', '1000268201_693b08cb0e.jpg\tA little girl climbing the stairs to her playhouse .\n', '1000268201_693b08cb0e.jpg\tA little girl in a pink dress going into a wooden cabin .\n']


In [7]:
from dataset import ImageCaptionDataset
from torchvision import transforms

dataset = ImageCaptionDataset(
    "data/images",
    "data/captions_fixed.txt",
    transform=transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.ToTensor(),
    ])
)


In [8]:
print(f"Dataset size: {len(dataset)}")

Dataset size: 3000


In [11]:
!python train.py


/content/drive/MyDrive/FinalProject/train.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  captions = [torch.tensor(caption, dtype=torch.long) for caption in captions]
Model checkpoints saved to data/encoder.pth and data/decoder.pth


In [16]:
!python evaluate.py

/content/drive/MyDrive/FinalProject/evaluate.py:52: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  encoder.load_state_dict(torch.load("/content/drive/MyDrive/FinalProject/enc